In [ ]:
## RUn in the main directory -> same as the main.py file

import numpy as np
import torch
from utils import load
from matplotlib import pyplot as plt
from scipy.spatial import distance
import seaborn as sns
import pickle as pkl

In [ ]:
modularity = [4,2,4]
arch = [4,48,48,48,4]
batch_size = 8
epochs = 20
lr = 0.05
seed = [1]
bn = 1.0
scheduler = 'exp'
experiment = 'train'
dev = load.load_device('0')
direct = './results/exemplar/dense/' + str(modularity) + '/' + str(arch) + '/' 

In [ ]:
####### EWP of paths dense NN
e = []
for s in seed:

    net = load.load_model(arch,False)(arch, 'relu', False).to(dev)
    net.load_state_dict(torch.load(direct + experiment + str(modularity) + str(lr) + str(batch_size) + str(epochs) + str(arch)  + str(bn) + '_Model' + str(s) + '.pth'))

    i=0
    for p in net.parameters():
        if len(p.data.size())!=1:
            if i == 0:
                ewp = np.transpose(p.data.abs().detach().cpu().numpy())
            else:
                ewp = np.matmul(ewp, np.transpose(p.data.abs().detach().cpu().numpy()))
            i+=1
    e.append(ewp)

ewp = np.zeros(e[0].shape)
for i in range(ewp.shape[0]):
    for j in range(ewp.shape[1]):
        p = []
        for k in range(len(seed)):
            p.append(e[k][i][j])
        ewp[i][j] = np.mean(p)

fig, ax = plt.subplots(figsize=(7,4))  
sns.heatmap(ewp, annot=True, fmt=".2f", annot_kws={"fontsize":14},yticklabels=np.arange(1,5),xticklabels=np.arange(1,5))
ewp = ewp
plt.xlabel('Output Unit', fontsize=13)
plt.ylabel('Input Unit', fontsize=13)
plt.suptitle('Edge Weight Product of paths dense NN \n Architecture :' + str(arch), fontsize=13)

import scipy

e1  = ewp[:2,:2].flatten()
e2  = ewp[:2,2:].flatten()

k1 =scipy.stats.ttest_ind(e2, e1, axis=0, equal_var=False, nan_policy='propagate', permutations=None, random_state=None, alternative='less', trim=0)

e1  = ewp[2:,:2].flatten()
e2  = ewp[2:,2:].flatten()

k2 =scipy.stats.ttest_ind(e1, e2, axis=0, equal_var=False, nan_policy='propagate', permutations=None, random_state=None, alternative='less', trim=0)
plt.title('t-statistic :' + str(round(k1[0], 2)) +', p-value :'+ str(round(k1[1], 2)) + '\nt-statistic :' + str(round(k2[0], 2)) +', p-value :'+ str(round(k2[1], 2)),  y = -0.3, fontsize=13)



plt.savefig('ewp_dense.png',facecolor='white',bbox_inches='tight')

In [ ]:
direct_models = './results/exemplar/edge/'  + str(modularity) + '/Models/' +str(arch) + '/' 
direct_results = './results/exemplar/edge/'  + str(modularity) + '/Results/' +str(arch) + '/' 
direct_masks = './results/exemplar/edge/'  + str(modularity) + '/Masks/' +str(arch) + '/' 

e = []
for s in seed: 
    pg = [5.0,2.5,2.0,1.5,1.0,0.5]
    fg = [1.0]

    pp = []
    ff = []
    ppg = []
    for i in range(len(pg)):
        for j in range(len(fg)):
            with open(direct_results+ 'prune_edgesboolean' + str(lr) +str(batch_size)+str(epochs) + str(None) + str(pg[i]) + str(None) + str(fg[j]) +'_Results' + str(s) +'.pkl', "rb") as fout:
                re = pkl.load(fout)
            pp.append(re[-1])
            ff.append(fg[j])
            ppg.append(pg[i])
    
    print(pg)
    print(pp)
    pruning_gradient = ppg[np.argmax(pp)]
    fn_factor = ff[np.argmax(pp)]
    prune_perc = np.max(pp)
    print('Shreyas Edge Pruning :', pruning_gradient, fn_factor, np.max(pp))

    model = load.load_model(arch,False)(arch, 'relu', False).to(dev)
    model.load_state_dict(torch.load(direct_models + 'prune_edgesboolean' + str(lr) +str(batch_size)+str(epochs) + str(None) + str(pruning_gradient) + str(None) + str(fn_factor) +'_Model' + str(s) +'.pth'))

    with open(direct_masks + 'prune_edgesboolean' + str(lr) +str(batch_size)+str(epochs) + str(None) + str(pruning_gradient) + str(None) + str(fn_factor) +'_Mask' + str(s) +'.pkl', "rb") as fout:
        weight_masks = pkl.load(fout)

    i=0
    for p in model.parameters():
        if len(p.data.size())!=1:
            print(p.data.size())
            if i == 0:
                ewp = np.transpose(p.data.abs().detach().cpu().numpy()*weight_masks[i].detach().cpu().numpy())
                paths = np.transpose(weight_masks[i].detach().cpu().numpy())
            else:
                ewp = np.matmul(ewp, np.transpose(p.data.abs().detach().cpu().numpy()*weight_masks[i].detach().cpu().numpy()))
                paths = np.matmul(paths, np.transpose(weight_masks[i].detach().cpu().numpy()))
            i+=1
    e.append(ewp)


In [ ]:
ewp = np.zeros(e[0].shape)
for i in range(ewp.shape[0]):
    for j in range(ewp.shape[1]):
        p = []
        for k in range(len(seed)):
            p.append(e[k][i][j])
        ewp[i][j] = np.mean(p)
fig, ax = plt.subplots(figsize=(7,4))  
sns.heatmap(ewp, annot=True, fmt=".2f", annot_kws={"fontsize":14},yticklabels=np.arange(1,5),xticklabels=np.arange(1,5))
ewp = ewp
plt.xlabel('Output Unit', fontsize=13)
plt.ylabel('Input Unit', fontsize=13)
plt.suptitle('Edge Weight Product of paths edge pruned NN \n Architecture :' + str(arch), fontsize=13)

import scipy
e1  = ewp[:2,:2].flatten()
e2  = ewp[:2,2:].flatten()

k1 =scipy.stats.ttest_ind(e2, e1, axis=0, equal_var=False, nan_policy='propagate', permutations=None, random_state=None, alternative='less', trim=0)

e1  = ewp[2:,:2].flatten()
e2  = ewp[2:,2:].flatten()

k2 =scipy.stats.ttest_ind(e1, e2, axis=0, equal_var=False, nan_policy='propagate', permutations=None, random_state=None, alternative='less', trim=0)
plt.title('t-statistic :' + str(round(k1[0], 2)) +', p-value :'+ str(round(k1[1], 2)) + '\nt-statistic :' + str(round(k2[0], 2)) +', p-value :'+ str(round(k2[1], 2)),  y = -0.3, fontsize=13)


plt.savefig('ewp_edge.png',facecolor='white',bbox_inches='tight')

In [ ]:
direct_models = './results/exemplar/unit_edge/' + str(modularity) + '/Models/' +str(arch) + '/' 
direct_results = './results/exemplar/unit_edge/' + str(modularity) + '/Results/' +str(arch) + '/' 
direct_masks = './results/exemplar/unit_edge/'+ str(modularity) + '/Masks/' +str(arch) + '/' 


e = []
for s in seed: 
    pruning_gradient = 2.5
    unit_pruning_gradient = 40.0
    fn_factor = 1.0

    model = load.load_model(arch,False)(arch, 'relu', False).to(dev)
    model.load_state_dict(torch.load(direct_models + 'pruneactivation_gradient_productboolean' + str(lr) +str(batch_size)+str(epochs)+ str(unit_pruning_gradient) + str(pruning_gradient) + str(None) + str(fn_factor) +'_Model' + str(s) + '.pth'))

    with open(direct_masks + 'pruneactivation_gradient_productboolean' + str(lr) +str(batch_size)+str(epochs)+ str(unit_pruning_gradient) + str(pruning_gradient) + str(None) + str(fn_factor) +'_Mask'+ str(s) + '.pkl', "rb") as fout:
        weight_masks = pkl.load(fout)

    i=0
    for p in model.parameters():
        if len(p.data.size())!=1:
            if i == 0:
                ewp = np.transpose(p.data.abs().detach().cpu().numpy()*weight_masks[i].detach().cpu().numpy())
                paths = np.transpose(weight_masks[i].detach().cpu().numpy())
            else:
                ewp = np.matmul(ewp, np.transpose(p.data.abs().detach().cpu().numpy()*weight_masks[i].detach().cpu().numpy()))
                paths = np.matmul(paths, np.transpose(weight_masks[i].detach().cpu().numpy()))
            i+=1
    e.append(ewp)

In [ ]:
ewp = np.zeros(e[0].shape)
for i in range(ewp.shape[0]):
    for j in range(ewp.shape[1]):
        p = []
        for k in range(len(seed)):
            p.append(e[k][i][j])
        ewp[i][j] = np.mean(p)
fig, ax = plt.subplots(figsize=(7,4))  
sns.heatmap(ewp, annot=True, fmt=".2f", annot_kws={"fontsize":14},yticklabels=np.arange(1,5),xticklabels=np.arange(1,5))
ewp = ewp#/np.sum(ewp)
plt.xlabel('Output Unit', fontsize=13)
plt.ylabel('Input Unit', fontsize=13)
plt.suptitle('Edge Weight Product of paths unit-edge pruned NN \n Architecture :' + str(arch), fontsize=13)

import scipy
e1  = ewp[:2,:2].flatten()
e2  = ewp[:2,2:].flatten()

k1 =scipy.stats.ttest_ind(e2, e1, axis=0, equal_var=False, nan_policy='propagate', permutations=None, random_state=None, alternative='less', trim=0)

e1  = ewp[2:,:2].flatten()
e2  = ewp[2:,2:].flatten()

k2 =scipy.stats.ttest_ind(e1, e2, axis=0, equal_var=False, nan_policy='propagate', permutations=None, random_state=None, alternative='less', trim=0)
plt.title('t-statistic :' + str(round(k1[0], 2)) +', p-value :'+ str(round(k1[1], 2)) + '\nt-statistic :' + str(round(k2[0], 2)) +', p-value :'+ str(round(k2[1], 2)),  y = -0.3, fontsize=13)



plt.savefig('ewp_unit_edge.png',facecolor='white',bbox_inches='tight')